In [ ]:
# Notebook from https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7
!pip install -q scipy langchain transformers playwright html2text sentence_transformers faiss-gpu
!pip install -q --upgrade git+https://github.com/huggingface/transformers

In [ ]:
!playwright install > /dev/null
!playwright install-deps > /dev/null

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

import nest_asyncio
#################################################################
# Tokenizer
#################################################################

model_name="mistralai/Mistral-7B-Instruct-v0.1"
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir="."
)
#load_in_4bits=True)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
article_a_indexer="https://www.droits-salaries.com/349927012-eurogerm/34992701200030-siege/T02121003194-accord-teletravail-2021-teletravail.shtml"

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = [article_a_indexer]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

# Converts HTML to plain text 
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=100)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [ ]:
# Create prompt template
prompt_template = """
### [INST] Instruction: Answer the question based on your french business agreements knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

In [ ]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
reponse= rag_chain.invoke("(days_of_remote_work_per_week:float, confidence_score:float) : S'il est fait mention d'une quotité par semaine et sans faire de calcul ni d'extrapolation à partir d'autres quotités, combien de jours maximum un salarié peut-il télétravailler par semaine ? Sinon, renvoyez-moi le tuple (None,0). Le chiffre doit être présent dans l'accord. Je veux exploiter vos réponses, donnez-moi les deux variables dans un tuple sans explication !")
print(reponse["text"])
reponse= rag_chain.invoke("(days_of_remote_work_per_month:float, confidence_score:float) : S'il est fait mention d'une quotité par mois et sans faire de calcul ni d'extrapolation à partir d'autres quotités, combien de jours maximum un salarié peut-il télétravailler par mois ? Sinon, renvoyez-moi le tuple (None,0). Le chiffre doit être présent dans l'accord. Je veux exploiter vos réponses, donnez-moi les deux variables dans un tuple sans explication !")
print(reponse["text"])
reponse= rag_chain.invoke("(days_of_remote_work_per_quarter:float, confidence_score:float) : S'il est fait mention d'une quotité par trimestre et sans faire de calcul ni d'extrapolation à partir d'autres quotités, combien de jours maximum un salarié peut-il télétravailler par trimestre ? Sinon, renvoyez-moi le tuple (None,0). Le chiffre doit être présent dans l'accord. Je veux exploiter vos réponses, donnez-moi les deux variables dans un tuple sans explication !")
print(reponse["text"])
reponse= rag_chain.invoke("(days_of_remote_work_per_year:float, confidence_score:float) : S'il est fait mention d'une quotité par année dans le context et sans faire de calcul ni d'extrapolation à partir d'autres quotités (hebdomadaire, mensuel, trimestriel), combien de jours maximum un salarié peut-il télétravailler par année ? Sinon, renvoyez-moi le tuple (None,0). Le chiffre doit être présent dans l'accord. Je veux exploiter vos réponses, donnez-moi les deux variables dans un tuple sans explication !")
print(reponse["text"])

In [ ]:
reponse= rag_chain.invoke("(days_of_remote_work_per_year:float, confidence_score:float) : If the context mentions a quota per year, and without calculating or extrapolating from other quotas (weekly, monthly, quarterly), how many days maximum can an employee telework per year? If not, send me the tuple (None,0). The figure must be present in the agreement. I want to exploit your answers, give me the two variables in a tuple without explanation!")
print(reponse["text"])